This is a simple combination of XY stage moving and FTS running. 
Choose x and y coordinates and set cosntants. 

Mira

In [4]:
%pylab
import serial
import time
import pickle
import sys
import zaber_commands as zc
import labjack_commands as lj
from XYStage import XYStage

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [5]:
phy_to_optical=zc.opt_per_physical
mm_per_step=zc.mm_per_step
acc_conversion=zc.acc_conversion
white_light_fringe=zc.white_light_fringe
max_speed=zc.max_speed
max_speed_o=zc.max_speed_o

In [3]:
x = raw_input('X-coordinate: ')
y = raw_input('Y-coordinate: ')
x = int(x)
y = int(y)
xy = XYStage()
xy.move(x,y)
#Make a note about coordinates/units?

X-coordinate: 1
Y-coordinate: 2


In [6]:
d = lj.init_labjack()
d

handle: None


NullHandleException: Couldn't open device. Please check that the device you are trying to open is connected.

In [7]:
#choose appropriate modem
dev = '/dev/tty.usbmodem1421'
#dev = '/dev/tty.usbmodem1411'
#dev = "COM3"
ser,err = zc.init_zaber(dev)

OSError: [Errno 2] No such file or directory: '/dev/tty.usbmodem1421'

In [ ]:
niter = 1
max_d = raw_input('Max_d: ') #mm
max_d = int(max_d)
speed = raw_input('Speed: ') #mm/s
speed = int(speed)
max_nu = 1000. # GHz
accel = 10000. # mm/s^2
oversample = 16
zc.clear_trigger_do_pos(ser,1)
zc.clear_trigger_do_pos(ser,3)
t=zc.clear_trigger_do_pos(ser,5)

In [10]:
d_accel=speed**2/(2.*accel)
startscan = white_light_fringe - max_d
stopscan = white_light_fringe + max_d
startpos = startscan - d_accel
stoppos = stopscan + d_accel

TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

In [11]:
zc.unpark(ser)
zc.set_do(ser,1,0)
zc.set_do(ser,2,0)
zc.set_accel(ser,2000.)
zc.set_max_speed(ser,max_speed_o)
zc.move_abs_wait(ser,0.2)
zc.home(ser)
zc.set_max_speed(ser,max_speed_o)
zc.move_abs_wait(ser,startpos)
zc.set_trigger_do_pos(ser,white_light_fringe,2,1,1)# set digital channel 2 with trigger 1
zc.set_trigger_do_pos(ser,startscan,1,3,1)
zc.set_trigger_do_pos(ser,stopscan,1,5,0)
zc.set_accel(ser,accel)
#zc.set_max_speed(ser,speed)
zc.set_lim_max(ser,stoppos)
zc.set_lim_min(ser,startpos)
zc.set_do(ser,1,0)
t=zc.set_do(ser,2,0)

NameError: name 'ser' is not defined

In [12]:
freq = lj.setup_scan(d,speed,max_nu,oversample,2)
scan_time = 2 * max_d/speed + 2 * speed/accel
n = int((1.68*scan_time - .2) * freq)
dx = speed / freq
print 'freq = ',freq, ' scan time = ',2*max_d/speed
print 'travel time = ',2 * max_d/speed, ' acc time = ',2 * speed/accel
print 'num samples requested = ',n
print 'max_nu in mm*-1 = ', max_nu/300.
data={'run':'test run'}
for iiter in np.arange(niter):
    #zc.move_abs(ser,stoppos)
    key=str(iiter)+'F'
    zc.move_vel(ser,speed)
    lj.start_adc_one(d,n,dx,data,key)
    print ' '
    print 'Iteration ',str(iiter)
    print 'size going forward = ',data['sig'+key].size
    time.sleep(.1)
    #zc.move_abs(ser,startpos)
    key=str(iiter)+'R'
    zc.move_vel(ser,-speed)
    lj.start_adc_one(d,n,-dx,data,key)
    print 'size going reverse = ',data['sig'+key].size
    time.sleep(.1)
zc.set_do(ser,1,0)
zc.set_do(ser,2,0)
zc.clear_trigger_do_pos(ser,1)
zc.clear_trigger_do_pos(ser,3)
zc.clear_trigger_do_pos(ser,5)
zc.set_accel(ser,2000.)
zc.set_max_speed(ser,max_speed_o)
zc.move_abs_wait(ser,white_light_fringe)
t=zc.park(ser)

NameError: name 'd' is not defined

In [13]:
print data['delay0F'].size
print data['sig0F'].size

NameError: name 'data' is not defined

In [14]:
figure(2)
#plot(data['delay0F'],data['scan0F'])
#plot(data['delay0F'],data['wlf0F'])
plot(data['sig0F'])
#plot(data['sig0R'])
#plot(data['delay0F'],data['sig0F'])
#plot(data['delay0R'],data['sig0R'])

figure(2)
#plot(data['delay0R'],data['scan0R'])
#plot(data['delay0R'],data['wlf0R'])
plot(data['delay0R'],data['sig0R'])

figure(3)
plot(data['wlf0F'])
plot(data['scan0F'])
plot(data['delay0R'],data['delay0F'])

NameError: name 'data' is not defined

In [15]:
sys.exit()

SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.
